In [40]:
import sys
!pip install biopython

In [41]:

from Bio import Entrez

In [42]:
Entrez.email = "lonphan@nih.gov" # user email
# API key from NCBI (https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/). 
# 10 queries per second a valid API key, otherwise 3 queries per seconds are allowed for 'None'
Entrez.api_key = None

# entrez query (term) can be build and test online using web query builder (https://www.ncbi.nlm.nih.gov/snp/advanced) 
# esearch handle
eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          term='LPL', # search for gene LPL
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )


In [43]:

# get esearch result
eSresult = Entrez.read(eShandle)

In [44]:
# review results 
for k in eSresult:
    print (k, ":", eSresult[k])
    
#Output: Web environment (&WebEnv) and query key (&query_key) parameters specifying the location on the Entrez history server of the list of UIDs matching the Entrez query
#https://www.ncbi.nlm.nih.gov/books/NBK25500/#chapter1.Storing_Search_Results
    

Count : 21205
TranslationStack : [DictElement({'Term': 'LPL[All Fields]', 'Field': 'All Fields', 'Count': '21205', 'Explode': 'N'}, attributes={}), 'GROUP']
IdList : ['1491562420', '1491463812', '1491435167', '1491412095', '1491387713', '1491361657', '1491332297', '1491304467', '1491179460', '1491137670', '1491081577', '1491070944', '1491033762', '1491015385', '1491004307', '1490990576', '1490964343', '1490911738', '1490880185', '1490867897']
TranslationSet : [DictElement({'To': 'LPL[All Fields]', 'From': 'LPL'}, attributes={})]
QueryKey : 1
RetMax : 20
RetStart : 0
WebEnv : NCID_1_194971829_130.14.22.215_9001_1547590318_789256681_0MetA0_S_MegaStore
QueryTranslation : LPL[All Fields]


In [45]:
# get result RSIDs list 'Idlist'
# total rs count 
rslist = (eSresult['IdList'])

In [46]:
# retmax = 20 so print only 20 RSIDs
# additional results can be retrieved by batches
# download in batches example http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139
for rs in rslist:
    print(rs)

1491562420
1491463812
1491435167
1491412095
1491387713
1491361657
1491332297
1491304467
1491179460
1491137670
1491081577
1491070944
1491033762
1491015385
1491004307
1490990576
1490964343
1490911738
1490880185
1490867897
